In [15]:
import pandas as pd

import pickle

from sklearn.metrics.pairwise import cosine_similarity

class CollabRecommendation:

    def __init__(self, user_data, product_data, rating_data):

        self.user_data = user_data

        self.product_data = product_data

        self.rating_data = rating_data
    def collaborative_filtering_recommendations(self, target_user_id, top_n=10):

        # Pivot the data to create a user-item matrix

        user_item_matrix = self.rating_data.pivot_table(

            index='userid',

            columns='productid',

            values='rating',

            aggfunc='mean'

        ).fillna(0)
        # Ensure target_user_id is an integer

        target_user_id = int(target_user_id)
        # Check if the target user exists in the matrix

        if target_user_id not in user_item_matrix.index:

            print(f"User ID {target_user_id} not found in data. Returning empty recommendations.")

            return []
        # Get the target user's ratings

        target_user_ratings = user_item_matrix.loc[target_user_id]
        # Identify items not yet rated by the user

        unrated_items = target_user_ratings[target_user_ratings == 0].index
        # Calculate average ratings for unrated items
        average_ratings = (

            self.rating_data[self.rating_data['productid'].isin(unrated_items)]

            .groupby('productid')['rating']

            .mean()

            .reset_index()

            .sort_values(by='rating', ascending=False)

        )

        # Merge with product details to include item names

        recommended_items_with_names = average_ratings.merge(

            self.product_data[['id', 'name']],

            left_on='productid',

            right_on='id',

            how='inner'

        )

        # Return the top N recommendations

        top_recommendations = recommended_items_with_names.head(top_n)

        return top_recommendations[['productid', 'name', 'rating']].values.tolist()

    def recommend(self, user_id, top_n=20):

        return self.collaborative_filtering_recommendations(user_id, top_n=top_n)

# Load the data

user_data = pd.read_csv('user_data.csv')

ratings_data = pd.read_csv('ratings_test.csv')

product_data = pd.read_csv('test_data.csv')

# Initialize the model

model = CollabRecommendation(user_data, product_data, ratings_data)



# Test with a user ID

user_id = '15980'

recommendations = model.recommend(user_id, top_n=10)

print(recommendations)